In [1]:
import pandas as pd
from pandabook import PandaBook, get_excel_tables, get_excel_table
import os
import numpy as np
import datetime
import uuid
from pathlib import Path

#### Saving  Pandas DataFrames to Excel Named Tables

In [2]:
book = PandaBook()

In [3]:
# Define the MultiIndex for columns
arrays = [
    ['A', 'A', 'B', 'B'],
    ['foo', 'bar', 'foo', 'bar']
]
multi_columns = pd.MultiIndex.from_arrays(arrays, names=['letter', 'type'])

# Create the DataFrame with MultiIndex columns
df_multi_col = pd.DataFrame(
    [[1, 2, 3, 4], [5, 6, 7, 8]],
    columns=multi_columns
)

# Define arrays for the MultiIndex
arrays = [
    ['A', 'A', 'B', 'B'],
    ['one', 'two', 'one', 'two']
]

# Create the MultiIndex
multi_rows = pd.MultiIndex.from_arrays(arrays, names=['letter', 'number'])

# Create the DataFrame
df_multi_row = pd.DataFrame({'value': [10, 20, 30, 40]}, index=multi_rows)


In [4]:
df_multi_col

letter   A       B    
type   foo bar foo bar
0        1   2   3   4
1        5   6   7   8

In [5]:
df_multi_row

value
letter number       
A      one        10
       two        20
B      one        30
       two        40

In [6]:
df_regular = df_multi_row.reset_index()

In [7]:
row_index = pd.MultiIndex.from_tuples(
    [('A', 1), ('A', 2), ('B', 1), ('B', 2)],
    names=['Group', 'Number']
)

# Define multi-level column index
col_index = pd.MultiIndex.from_tuples(
    [('Score', 'Math'), ('Score', 'Science'), ('Grade', 'Math'), ('Grade', 'Science')],
    names=['Category', 'Subject']
)

# Create the DataFrame
df_multi_indexed = pd.DataFrame(
    np.random.randint(50, 100, size=(4, 4)),
    index=row_index,
    columns=col_index
)
df_multi_indexed

Category     Score         Grade        
Subject       Math Science  Math Science
Group Number                            
A     1         55      52    90      60
      2         93      75    61      61
B     1         71      64    73      57
      2         73      56    92      98

In [8]:
data = {
    'uuid4_col': [uuid.uuid4() for _ in range(3)],
    'list_col': [[1, 2, 3], [4, 5], [6]],
    'dict_col': [{'a': 1, 'b': 2}, {'c': 3}, {'d': 4, 'e': 5}],
    'datetime_col': [datetime.datetime(2024, 6, 1, 12, 0), datetime.datetime(2024, 6, 2, 13, 30), datetime.datetime(2024, 6, 3, 15, 45)],
    'date_col': [datetime.datetime(2024, 6, 1), datetime.datetime(2024, 6, 2), datetime.datetime(2024, 6, 3)],
    'severity' : ['High', 'Medium', 'Low'],
    'long_text' : 'very long text ' * 25,
    'booleans' : [True, False, True]
}

df_oddvalues = pd.DataFrame(data)

In [9]:
df_oddvalues

,uuid4_col,list_col,dict_col,datetime_col,date_col,severity,long_text,booleans
0,33f8919a-e44a-4372-9663-8e690d050cc1,"[1, 2, 3]","{'a': 1, 'b': 2}",2024-06-01 12:00:00,2024-06-01,High,very long text very long text very long text v...,True
1,c6d5c8f9-265e-4858-876a-dea105a1d44f,"[4, 5]",{'c': 3},2024-06-02 13:30:00,2024-06-02,Medium,very long text very long text very long text v...,False
2,1516d5ca-798a-46fb-a6d5-8c857bcec742,[6],"{'d': 4, 'e': 5}",2024-06-03 15:45:00,2024-06-03,Low,very long text very long text very long text v...,True


In [10]:
book.add_table(
    df=df_multi_col,
    name='df_multi_col',
    index=True,
    description='Sample data with a multi-level column index'
    )

book.add_table(
    df=df_multi_row,
    name='df_multi_row',
    index=True,
        description='Sample data with a multi-level row index'
    )


book.add_table(
    df=df_regular,
    name='df_regular',
    index=True,
    description='Sample data with a regular index'
    )

book.add_table(
    df=df_oddvalues,
    name='df_oddval',
    index=False,
    description='Sample data with various datatypes, and conditional formatting',
    columns={
        'uuid4_col' : {'is_pk' : True, 'width' : 20},
        'severity' :  {'conditional_style' :{
            'Low' : 'Good',
            'Medium' : 'Neutral',
            'High' : 'Bad'
            }
        }
    }
)

book.add_table(
    df=df_multi_indexed,
    name='df_multi_indexed',
    index=True,
    description='Multi indexed'
    )

In [11]:
output_folder = Path('output')
output_file = output_folder / 'example_output.xlsx'

book.save(output_file)
os.startfile(output_file)

#### Loading Excel Named Tables to Pandas DataFrames

In [12]:
# Get all named tables from a file into a dictionary
all_data = get_excel_tables(output_file, convert_nan_to_none=True)
all_data.keys()

dict_keys(['index', 'df_multi_col', 'df_multi_row', 'df_regular', 'df_oddval', 'df_multi_indexed'])

In [13]:
all_data['df_regular']

,index,letter,number,value
0,0,A,one,10
1,1,A,two,20
2,2,B,one,30
3,3,B,two,40


In [14]:
# Alternatively, just load a specific table
get_excel_table(filename=output_file, table_name='df_multi_row')

,letter,number,value
0,A,one,10
1,A,two,20
2,B,one,30
3,B,two,40


Please note the following limitations when loading data:

* Data types that were converted to strings (eg lists, dictionary) are not automatically deserialized into python data types
* Multi-level indexes are not recontstructed